In [ ]:
import pandas as pd

In [ ]:
file_path0 = "/content/train_data.txt"
file_path1 = "/content/test_data.txt"
file_path2 = "/content/test_data_solution.txt"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


Step 1 : Load and Parse Training data.

In [ ]:
with open(file_path0, "r", encoding="utf-8") as file:
    train_lines = file.readlines()

Parse the training data

In [ ]:
train_data = []
for line in train_lines:
    parts = line.strip().split(" ::: ")
    train_data.append({
        "ID" : parts[0],
        "TITLE" : parts[1],
        "GENRE" : parts[2],
        "DESCRIPTION" : parts[3],
    })

Convert to DataFrame

In [ ]:
train_df = pd.DataFrame(train_data)

Features and target for training

In [ ]:
X_train = train_df["DESCRIPTION"]
Y_train = train_df["GENRE"]

Step 2 : Load and parse testing data

In [ ]:
with open(file_path1, "r", encoding="utf-8") as file:
    test_lines = file.readlines()

Parse testing data

In [ ]:
test_data = []
for line in test_lines:
    parts = line.strip().split(" ::: ")
    test_data.append({
        "ID" : parts[0],
        "TITLE" : parts[1],
        "DESCRIPTION" : parts[2],
    })

Convert to DataFrame

In [ ]:
test_df = pd.DataFrame(test_data)

In [ ]:
X_test = test_df["DESCRIPTION"]

Step 3 : Vectorize the text data

In [ ]:
vectorizer = CountVectorizer()

In [ ]:
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

Step 4 : Train the model

In [ ]:
lr = LogisticRegression()
lr.fit(X_train_vectorized, Y_train)

Step 5 : Make Predictions on testing data

In [ ]:
y_pred = lr.predict(X_test_vectorized)
print(y_pred)

Step 6 : Load and use the test solution for data evaluation

In [ ]:
with open(file_path2, "r", encoding="utf-8") as file:
    test_sol_lines = file.readlines()

Parse the test solution data

In [ ]:
test_sol_data = []
for line in test_sol_lines:
    parts = line.strip().split(" ::: ")
    test_sol_data.append(parts[2])

Convert to Series

In [ ]:
Y_test = pd.Series(test_sol_data)

Step 7 : Evaluate the model

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
print("Accuracy :",accuracy_score(Y_test, y_pred))
print("Confusion_Matrix : \n" , confusion_matrix(Y_test, y_pred))